In [437]:

import pandas as pd
import numpy as np

cars_raw = pd.read_csv("otomoto_cars_parsed.csv", low_memory=False)
cars = cars_raw.copy()

In [438]:
cars.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224835 entries, 0 to 224834
Data columns (total 23 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   Listing_URL                          224835 non-null  object
 1   Make                                 224835 non-null  object
 2   Model                                224830 non-null  object
 3   Body_Type                            224835 non-null  object
 4   Fuel_Type                            224835 non-null  object
 5   Gearbox                              224791 non-null  object
 6   Transmission                         210394 non-null  object
 7   Year                                 224835 non-null  int64 
 8   Mileage                              224835 non-null  int64 
 9   Engine_Capacity                      224835 non-null  int64 
 10  Engine_Power                         224835 non-null  int64 
 11  No_Accidents              

In [439]:
# Define columns to analyze and convert
boolean_columns = ['No_Accidents', 'Service_Record', 
                   'Is_Imported', 'First_Owner']

# Print original summaries
print("=== ORIGINAL DATA SUMMARIES ===")
for col in boolean_columns:
    summary = cars.groupby(col, dropna=False).size()
    print(f"{col}:")
    print(summary)
    print()

# Convert all columns
for col in boolean_columns:
    cars[col] = np.where(cars[col] == 'Tak', 1, 0)
    cars[col] = cars[col].astype(int)

# Print converted summaries
print("=== CONVERTED DATA SUMMARIES ===")
for col in boolean_columns:
    summary = cars.groupby(col, dropna=False).size()
    print(f"{col}:")
    print(summary)
    print()

=== ORIGINAL DATA SUMMARIES ===
No_Accidents:
No_Accidents
Tak    127281
NaN     97554
dtype: int64

Service_Record:
Service_Record
Tak    113551
NaN    111284
dtype: int64

Is_Imported:
Is_Imported
Nie     99681
Tak    124713
NaN       441
dtype: int64

First_Owner:
First_Owner
Tak     66675
NaN    158160
dtype: int64

=== CONVERTED DATA SUMMARIES ===
No_Accidents:
No_Accidents
0     97554
1    127281
dtype: int64

Service_Record:
Service_Record
0    111284
1    113551
dtype: int64

Is_Imported:
Is_Imported
0    100122
1    124713
dtype: int64

First_Owner:
First_Owner
0    158160
1     66675
dtype: int64



In [440]:
summary = cars.groupby('Seller_Type', dropna=False).size()
print(summary)

Seller_Type
PRIVATE         133889
PROFESSIONAL     90935
NaN                 11
dtype: int64


In [441]:
cars = cars.dropna(subset=['Seller_Type'])
cars['Seller_Type'] = np.where(cars['Seller_Type'] == 'PROFESSIONAL', 1, 0)
cars['Seller_Type'] = cars['Seller_Type'].astype(int)
cars.rename(columns={'Seller_Type': 'Professional_Seller'}, inplace=True)

In [442]:
summary = cars.groupby('Professional_Seller', dropna=False).size()
print(summary)

Professional_Seller
0    133889
1     90935
dtype: int64


In [443]:
summary = cars.groupby('Transmission', dropna=False).size()
print(summary)

Transmission
4x4 (dołączany automatycznie)     20664
4x4 (dołączany ręcznie)            4023
4x4 (stały)                       27604
Na przednie koła                 140640
Na tylne koła                     17452
NaN                               14441
dtype: int64


In [444]:
# Remove rows with missing values in the Transmission column
cars = cars.dropna(subset=['Transmission'])

In [445]:
# Mapowanie uproszczonych kategorii napędu
def simplify_transmission(value):
    if value == 'Na przednie koła':
        return 'FWD'
    elif value == 'Na tylne koła':
        return 'RWD'
    elif isinstance(value, str) and value.startswith('4x4'):
        return 'AWD'
    else:
        return 'Unknown'  # albo np. np.nan

# Utwórz nową kolumnę z uproszczonymi kategoriami
cars['Transmission'] = cars['Transmission'].apply(simplify_transmission)

# Sprawdź wyniki
print(cars['Transmission'].value_counts())

Transmission
FWD    140640
AWD     52291
RWD     17452
Name: count, dtype: int64


In [446]:
# Remove cars with price lower than 1000
cars = cars[cars['Price'] >= 1000]

In [447]:
# Remove cars with price higher than 3000000
cars = cars[cars['Price'] <= 3000000]

In [448]:
# Remove rows with missing values in the Gearbox column
cars = cars.dropna(subset=['Gearbox'])

In [449]:
# Usuń wiersze, w których model to 'inne' 
cars = cars[~cars['Model'].str.lower().eq('inny')]


In [450]:
# Create temporary combined column
cars['Make_Model'] = cars['Make'].astype(str) + ' ' + cars['Model'].astype(str)

# Count occurrences
make_model_counts = cars['Make_Model'].value_counts()

# Keep only combinations with at least 20 listings
valid_make_models = make_model_counts[make_model_counts >= 20].index

# Filter dataset
cars = cars[cars['Make_Model'].isin(valid_make_models)].copy()

# Drop temporary column
cars.drop(columns=['Make_Model'], inplace=True)

# Output size
print(f"Remaining rows after filtering rare Make-Model combinations: {len(cars)}")

Remaining rows after filtering rare Make-Model combinations: 206584


In [451]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
summary = cars.groupby('Model', dropna=False).size()
print(summary)

Model
1007                       22
107                        84
108                        45
126                        61
147                        22
1500                      175
159                       135
2                         279
2008                      979
206                        91
206 CC                     22
206 plus                   30
207                       388
207 CC                     52
208                       860
3                         766
3008                     1439
300C                       40
301                        46
307                       134
307 CC                     39
308                      1494
308 CC                     28
370 Z                      22
3GT                       188
4008                       20
407                       173
408                        30
5                         159
500                       768
5008                      694
500L                      142
500X                      223
500e

In [452]:
# Set display options to show all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 200)
cars.head(10)

,Listing_URL,Make,Model,Body_Type,Fuel_Type,Gearbox,Transmission,Year,Mileage,Engine_Capacity,Engine_Power,No_Accidents,Service_Record,Is_Imported,First_Owner,Professional_Seller,Full_Description,Equipment_Audio_and_Multimedia,Equipment_Comfort_and_Extras,Equipment_Driver_Assistance_Systems,Equipment_Performance_and_Tuning,Equipment_Safety,Price
0,https://www.otomoto.pl/osobowe/oferta/volkswagen-golf-gti-tylko-150tys-km-klimatronic-szyberdach-stan-perfekt-ID6HwhaX.html,Volkswagen,Golf,Kompakt,Benzyna,Manualna,FWD,2012,150000,1984,210,1,1,1,0,0,Volkswagen Golf VI 2.0 TSI GTI NUMER TELEFONU: ==VW GOLF 6 GTI== 2.0 TSI 211 KM ŚWIEŻO SPROWADZONY Z NIEMIEC GOTOWY DO REJESTRACJI-PO WSZYSTKICH OPŁATACH CELNO SKARBOWYCH. Auto w perfekcyjnym stan...,"[""Radio""]","[""Klimatyzacja automatyczna, dwustrefowa"", ""Tapicerka materiałowa"", ""Podgrzewany fotel kierowcy"", ""Podgrzewany fotel pasażera"", ""Sportowe fotele - przód"", ""Podłokietniki - przód"", ""Podłokietniki -...","[""Kontrola odległości z przodu (przy parkowaniu)"", ""Kontrola odległości z tyłu (przy parkowaniu)"", ""Podgrzewane lusterka boczne"", ""Kontrola trakcji"", ""Spryskiwacze reflektorów"", ""Lampy doświetlają...","[""Zawieszenie sportowe""]","[""ABS"", ""ESP"", ""System wspomagania hamowania"", ""Poduszka powietrzna kierowcy"", ""Poduszka powietrzna pasażera"", ""Poduszka kolan pasażera"", ""Kurtyny powietrzne - przód"", ""Boczne poduszki powietrzne ...",39900
1,https://www.otomoto.pl/osobowe/oferta/opel-mokka-cosmo-szyber-dach-stan-bdb-nowy-rozrzad-i-hamulce-vat-marza-ID6HuM0n.html,Opel,Mokka,SUV,Diesel,Manualna,FWD,2013,151200,1686,130,1,1,1,1,0,"Opel Mokka 1.7 CDTI Cosmo S&S 4x4 Do sprzedania Opel Mokka Cosmo jedna z najwyższych wersji wyposażona w legendarny nie zniszczalny silnik diesla 1,7 cdti , bardzo znana jednostka słynąca z przebi...","[""Interfejs Bluetooth"", ""Radio"", ""Zestaw głośnomówiący"", ""Gniazdo USB"", ""System nawigacji satelitarnej"", ""Dostęp do internetu""]","[""Klimatyzacja automatyczna"", ""Tapicerka częściowo skórzana"", ""Elektrycznie ustawiany fotel kierowcy"", ""Podgrzewany fotel kierowcy"", ""Podgrzewany fotel pasażera"", ""Regul. elektr. podparcia lędźwio...","[""Tempomat"", ""Lampy ksenonowe"", ""Kontrola odległości z przodu (przy parkowaniu)"", ""Kontrola odległości z tyłu (przy parkowaniu)"", ""Park Assistant - asystent parkowania"", ""Lusterka boczne ustawiane...","[""Felgi aluminiowe 18""]","[""ABS"", ""ESP"", ""System powiadamiania o wypadku"", ""Poduszka powietrzna kierowcy"", ""Poduszka powietrzna pasażera"", ""Kurtyny powietrzne - przód"", ""Boczna poduszka powietrzna kierowcy"", ""Boczne podusz...",32900
2,https://www.otomoto.pl/osobowe/oferta/hyundai-ix35-hyundai-ix35-1-6-benzyna-skory-nawigacja-kamera-ID6HwlDg.html,Hyundai,ix35,SUV,Benzyna,Manualna,FWD,2015,125000,1591,135,1,1,1,1,0,"Hyundai ix35 1.6 2WD Comfort Rok produkcji: 2015 Silnik: 1.6 BENZYNA Skrzynia: MANUALNA Moc silnika: 135 KM CENA: 45 900zł Przebieg: 125 tys km Samochod w bardzo dobrym stanie, dzieki dobrym param...","[""Interfejs Bluetooth"", ""Radio"", ""Zestaw głośnomówiący"", ""Gniazdo USB"", ""System nawigacji satelitarnej"", ""System nagłośnienia"", ""Ekran dotykowy""]","[""Klimatyzacja automatyczna, dwustrefowa"", ""Tapicerka częściowo skórzana"", ""Podgrzewany fotel kierowcy"", ""Podgrzewany fotel pasażera"", ""Podłokietniki - przód"", ""Podłokietniki - tył"", ""Kierownica s...","[""Podgrzewane lusterka boczne"", ""Kontrola trakcji"", ""Automatyczna kontrola zjazdu ze stoku"", ""Wspomaganie ruszania pod górę- Hill Holder"", ""Wspomaganie kierownicy""]","[""Felgi aluminiowe 16""]","[""ABS"", ""ESP"", ""Elektroniczny system rozdziału siły hamowania"", ""System wspomagania hamowania"", ""Poduszka powietrzna kierowcy"", ""Poduszka powietrzna pasażera"", ""Kurtyny powietrzne - przód"", ""Boczn...",45900
3,https://www.otomoto.pl/osobowe/oferta/kia-ceed-kia-ceed-1-6-benzyna-nawigacja-kamera-ID6HwlG9.html,Kia,Ceed,Kombi,Benzyna,Manualna,FWD,2015,145000,1591

In [453]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
summary = cars.groupby('Model', dropna=False).size()
print(summary)

Model
1007                       22
107                        84
108                        45
126                        61
147                        22
1500                      175
159                       135
2                         279
2008                      979
206                        91
206 CC                     22
206 plus                   30
207                       388
207 CC                     52
208                       860
3                         766
3008                     1439
300C                       40
301                        46
307                       134
307 CC                     39
308                      1494
308 CC                     28
370 Z                      22
3GT                       188
4008                       20
407                       173
408                        30
5                         159
500                       768
5008                      694
500L                      142
500X                      223
500e

In [454]:
# Count the number of cars with Price < 1000
num_cars_under_1000 = cars[cars['Price'] <1000].shape[0]

print(f"Number of cars with price lower than 100: {num_cars_under_1000}")

Number of cars with price lower than 100: 0


In [455]:
# Count the occurrences of each model
model_counts = cars['Make'].value_counts()

# Identify models with fewer than 10 occurrences
rare_models = model_counts[model_counts < 10].index

# Filter rows where the model is one of the rare ones
rare_model_rows = cars[cars['Make'].isin(rare_models)]

# Display the result
rare_model_rows

,Listing_URL,Make,Model,Body_Type,Fuel_Type,Gearbox,Transmission,Year,Mileage,Engine_Capacity,Engine_Power,No_Accidents,Service_Record,Is_Imported,First_Owner,Professional_Seller,Full_Description,Equipment_Audio_and_Multimedia,Equipment_Comfort_and_Extras,Equipment_Driver_Assistance_Systems,Equipment_Performance_and_Tuning,Equipment_Safety,Price


In [456]:
# Zdefiniuj mapowanie
fuel_mapping = {
    'Benzyna': 'Petrol-based',
    'Benzyna+LPG': 'Petrol-based',
    'Benzyna+CNG': 'Petrol-based',
    'Diesel': 'Diesel',
    'Hybryda': 'Hybrid',
    'Hybryda Plug-in': 'Hybrid',
    'Elektryczny': 'Electric'
    # 'Etanol' i 'Wodór' nie są uwzględnione => będą jako NaN
}

# Zastosuj mapowanie
cars['Fuel_Type'] = cars['Fuel_Type'].map(fuel_mapping)

# Usuń wiersze, które nie zostały przypisane do żadnej grupy (czyli Etanol, Wodór, ewentualnie NaN)
cars = cars[~cars['Fuel_Type'].isna()]

# Policz liczebność po grupowaniu
fuel_counts = cars['Fuel_Type'].value_counts()

print(fuel_counts)

Fuel_Type
Petrol-based    110891
Diesel           80285
Hybrid           13169
Electric          2217
Name: count, dtype: int64


In [457]:
summary = cars.groupby('Fuel_Type', dropna=False).size()
print(summary)

Fuel_Type
Diesel           80285
Electric          2217
Hybrid           13169
Petrol-based    110891
dtype: int64


In [458]:
# Utworzenie nowych zmiennych
cars['Log_Price'] = np.log1p(cars['Price'])
cars['Log_Mileage'] = np.log1p(cars['Mileage'])
from datetime import datetime

current_year = datetime.now().year
cars['Age'] = current_year - cars['Year']
cars.drop(columns=['Year'], inplace=True)
# Usuń samochody z przyszłości (wiek < 0)
cars = cars[cars["Age"] >= 0]
# Stworzenie zmiennej log(Age) — zabezpieczone przed log(0)
cars['Log_Age'] = np.log1p(cars['Age'])
cars['Mileage_per_Year'] = cars['Mileage'] / (cars['Age'] + 1)
# Tworzenie nowej zmiennej: moc na litr, pomijając auta bez tradycyjnego silnika spalinowego
cars['Power_per_Liter'] = np.where(
    cars['Engine_Capacity'] > 0,
    cars['Engine_Power'] / (cars['Engine_Capacity'] / 1000),
    np.nan  # dla elektrycznych lub błędnych danych
)

In [459]:
def save_processed_data(df, filename='otomoto_cars_parsed2.csv'):
    try:
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Processed data saved to {filename}")
        return True
    except Exception as e:
        print(f"Error saving processed data: {e}")
        return False
save_processed_data(cars, 'otomoto_cars_parsed2.csv')

Processed data saved to otomoto_cars_parsed2.csv


True